In [1]:
import pandas as pd 
from collections import defaultdict

In [2]:
df=pd.DataFrame.from_csv("../../data/duolingo/es_en.slam.20171218.train.csv")
len(df.index)

/Users/hsr/envs/rl/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/Users/hsr/envs/rl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


1968555

In [3]:
user_history=df[['token','part_of_speech','correctness','user']].groupby(['user']).agg({'token':lambda x: tuple(x), 
                                   'part_of_speech':lambda x: tuple(x),'correctness':lambda x:tuple(x)})


In [5]:
def combine_train_valid_test_sheets(path):
    train_df=pd.DataFrame.from_csv(path)
    valid_df=pd.DataFrame.from_csv(path.replace("train","dev"))
    test_df=pd.DataFrame.from_csv(path.replace("train","test"))
    print(len(train_df.columns))
    print(train_df.columns)
    print(len(valid_df.columns))
    print(len(test_df.columns))
    print(set(train_df.columns).difference(set(valid_df.columns)))
    print(set(train_df.columns).difference(set(test_df.columns)))
    frames = [train_df, valid_df, test_df]
    result = pd.concat(frames,keys=['train','dev','test'],names=['from', 'idx'])
    return result
    
result=combine_train_valid_test_sheets("../../data/duolingo/es_en.slam.20171218.train.csv")
concat_result=result.reset_index(level=['from']).reset_index(drop=True)
from collections import defaultdict
user_list=list(concat_result['user'].unique())
user2id=defaultdict(lambda:len(user2id))
with open("user_id_mappingex.txt","w") as f:
    for user in user_list:
        f.write(user+"\t"+str(user2id[user])+"\n")
    

/Users/hsr/envs/rl/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/Users/hsr/envs/rl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/hsr/envs/rl/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hsr/envs/rl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Colum

32
Index(['user', 'countries', 'days', 'client', 'session', 'format', 'time',
       'session_id', 'exercise_id', 'token_id', 'token', 'part_of_speech',
       'morphological_features_Definite', 'morphological_features_Gender',
       'morphological_features_Number', 'morphological_features_PronType',
       'morphological_features_fPOS', 'dependency_label',
       'dependency_edge_head', 'correctness', 'morphological_features_Mood',
       'morphological_features_Person', 'morphological_features_Tense',
       'morphological_features_VerbForm', 'morphological_features_Case',
       'morphological_features_PrepCase', 'morphological_features_Poss',
       'morphological_features_NumType', 'morphological_features_Reflex',
       'morphological_features_Foreign', 'morphological_features_Degree',
       'morphological_features_Polite'],
      dtype='object')
32
32
set()
set()


In [6]:
directory = "../../data/duolingo/"
pos = "part_of_speech"
token = "token"

skill2id = {'unk':1}

count = 1
with open(directory+"unigram_pos_train1.csv","w") as f:
    for idx, row in user_history.iterrows():
        total = len(row['part_of_speech'])
        pos_tags = row['part_of_speech']
        tokens = row['token']
        train_skills_list = []
        for i in range(total):
            key = pos_tags[i] + tokens[i]
            if key in skill2id:
                train_skills_list.append(str(skill2id[key]))
            else:
                count += 1
                skill2id[key] = count
#                 print(str(skill2id[key]))
                train_skills_list.append(str(skill2id[key]))
        
        train_correctness_list=[str(int(value)) for value in list(row['correctness'])]
#         import pdb; pdb.set_trace()
        f.write(str(len(train_skills_list))+" # "+str(user2id[idx])+"\n"+str(",".join(train_skills_list))+"\n"+",".join(train_correctness_list)+"\n")
        
with open(directory + "unigram_pos_id_mapping.txt","w") as f:
    for skill in skill2id:
        f.write(skill+"\t"+str(skill2id[skill])+"\n")

In [7]:
df=pd.DataFrame.from_csv("../../data/duolingo/es_en.slam.20171218.dev.csv")

user_history=df[['token','part_of_speech','correctness','user']].groupby(['user']).agg({'token':lambda x: tuple(x), 
                                   'part_of_speech':lambda x: tuple(x),'correctness':lambda x:tuple(x)})

/Users/hsr/envs/rl/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.
/Users/hsr/envs/rl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [8]:
directory = "../../data/duolingo/"

count = 0
pos_count = 0
with open(directory+"unigram_pos_dev1.csv","w") as f:
    for idx, row in user_history.iterrows():
        total = len(row['part_of_speech'])
        pos_tags = row['part_of_speech']
        tokens = row['token']
        skills_list = []
        for i in range(total):
            key = pos_tags[i] + tokens[i]
            if key in skill2id:
                skills_list.append(str(skill2id[key]))
                pos_count += 1
            else:
                skills_list.append('1')
                count += 1
        
        correctness_list=[str(int(value)) for value in list(row['correctness'])]
        f.write(str(len(skills_list))+" # "+str(user2id[idx])+"\n"+str(",".join(str(skills_list)))+"\n"+",".join(correctness_list)+"\n")
    print(count)
    print(pos_count)

476
270345


In [ ]:
df=pd.DataFrame.from_csv("../../data/duolingo/es_en.slam.20171218.test.csv")

user_history=df[['token','part_of_speech','correctness','user']].groupby(['user']).agg({'token':lambda x: tuple(x), 
                                   'part_of_speech':lambda x: tuple(x),'correctness':lambda x:tuple(x)})

In [ ]:
directory = "../../data/duolingo/"

count = 0
pos_count = 0
with open(directory+"unigram_pos_test.csv","w") as f:
    for idx, row in user_history.iterrows():
        total = len(row['part_of_speech'])
        pos_tags = row['part_of_speech']
        tokens = row['token']
        skills_list = []
        for i in range(total):
            key = pos_tags[i] + tokens[i]
            if key in skill2id:
                skills_list.append(str(skill2id[key]))
                pos_count += 1
            else:
                skills_list.append('1')
                count += 1
        
        correctness_list=[str(int(value)) for value in list(row['correctness'])]
        f.write(str(len(skills_list))+" # "+str(user2id[idx])+"\n"+str(",".join(str(skills_list)))+"\n"+",".join(correctness_list)+"\n")
    print(count)
    print(pos_count)